In [18]:
import pandas as pd
import numpy as np
import joblib
import os
import benchmark_common as bcommon
import config as cfg
import benchmark_test as btest
import argparse
import tools.funclib as funclib
import tools.embedding_esm as esmebd
import time
import benchmark_evaluation as eva
from pandarallel import pandarallel #  import pandaralle
pandarallel.initialize() 

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 104 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
